In [1]:
import os
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import os 
import pandas as pd
import numpy as np
import random
import pickle
import pandas as pd
from Bio import SeqIO
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score
# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax

from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import roc_curve

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import load_model
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import imblearn
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, classification_report

2024-09-14 22:42:54.566340: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
os.chdir("/homes/t326h379/OGalNAc/Ankh_154_Proteins")

Training_Protein = []
with open("One_hundred_Thirty_Seven_Training_Proteins_OGalNAc.txt") as Subash:
    for line in Subash:
        Training_Protein.append(line.strip("\n"))
        
os.chdir("/home/t326h379/OGalNAc")

df_positive_positive = pd.read_csv('Positive_OGalNAc_ProtT5_Positive_Features_vectors_1837_Net_O_Glyc_4.txt',header=None)              
df_negative_negative = pd.read_csv('Negative_OGalNAc_ProtT5_Negative_Features_vectors_59414_Net_O_Glyc_4.txt',header=None)  

Header_name = ["Position","PID","S_or_T"]+[i for i in range(1,1025)]
df_positive_positive.columns = Header_name
df_negative_negative.columns = Header_name

df_positive = df_positive_positive[df_positive_positive['PID'].isin(Training_Protein)]
df_negative = df_negative_negative[df_negative_negative['PID'].isin(Training_Protein)]

# Multi Layer Perceptron 10 Fold CV

In [3]:
from imblearn.under_sampling import RandomUnderSampler
a = random.sample(range(1, 1000000), 20)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        y_train_1 = tf.keras.utils.to_categorical(y_train,2)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        model = Sequential()

        model.add(Input(shape=(1024,)))

        model.add(Dense(64,activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(2,activation='softmax'))



        model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])

        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                        monitor = 'val_accuracy',
                                        verbose=0, 
                                        save_weights_only=False,
                                        save_best_only=True)

        reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=5, verbose=1, min_delta=1e-4, mode='max')

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

        history = model.fit(X_train, y_train_1, epochs=21,verbose=1, batch_size=32)

        Y_pred = model.predict(xval)
        Y_pred = (Y_pred > 0.5)
        y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
        y_pred = np.array(y_pred)

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("MLP Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(528, 1024) (49, 1024)
(14419, 1024) (1083, 1024)
Training Dataset
(1056, 1024) (1056,)
Validation Dataset
(98, 1024) (98,)




Epoch 1/21
33/33 [==============================] - 1s 1ms/step - loss: 0.6053 - accuracy: 0.7045
Epoch 2/21
33/33 [==============================] - 0s 1ms/step - loss: 0.4811 - accuracy: 0.7869
Epoch 3/21
33/33 [==============================] - 0s 1ms/step - loss: 0.4038 - accuracy: 0.8352
Epoch 4/21
33/33 [==============================] - 0s 1ms/step - loss: 0.3453 - accuracy: 0.8636
Epoch 5/21
33/33 [==============================] - 0s 1ms/step - loss: 0.3101 - accuracy: 0.8854
Epoch 6/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2777 - accuracy: 0.9062
Epoch 7/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2431 - accuracy: 0.9167
Epoch 8/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2263 - accuracy: 0.9167
Epoch 9/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1997 - accurac

33/33 [==============================] - 0s 1ms/step - loss: 0.3218 - accuracy: 0.8831
Epoch 6/21
33/33 [==============================] - 0s 1ms/step - loss: 0.3019 - accuracy: 0.8859
Epoch 7/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2721 - accuracy: 0.8926
Epoch 8/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2445 - accuracy: 0.9125
Epoch 9/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2200 - accuracy: 0.9230
Epoch 10/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2070 - accuracy: 0.9221
Epoch 11/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1962 - accuracy: 0.9344
Epoch 12/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1612 - accuracy: 0.9515
Epoch 13/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1558 - accuracy: 0.9544
Epoch 14/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1489 - accuracy: 0.9610
Epoch 15/21
33/33 [=======

34/34 [==============================] - 0s 1ms/step - loss: 0.2153 - accuracy: 0.9329
Epoch 11/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2093 - accuracy: 0.9395
Epoch 12/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1907 - accuracy: 0.9546
Epoch 13/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1767 - accuracy: 0.9480
Epoch 14/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1584 - accuracy: 0.9537
Epoch 15/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1474 - accuracy: 0.9584
Epoch 16/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1341 - accuracy: 0.9641
Epoch 17/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1198 - accuracy: 0.9716
Epoch 18/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1192 - accuracy: 0.9698
Epoch 19/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1134 - accuracy: 0.9735
Epoch 20/21
34/34 [===

34/34 [==============================] - 0s 1ms/step - loss: 0.1343 - accuracy: 0.9636
Epoch 14/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1264 - accuracy: 0.9654
Epoch 15/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1127 - accuracy: 0.9776
Epoch 16/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1061 - accuracy: 0.9710
Epoch 17/21
34/34 [==============================] - 0s 1ms/step - loss: 0.0953 - accuracy: 0.9757
Epoch 18/21
34/34 [==============================] - 0s 1ms/step - loss: 0.0879 - accuracy: 0.9804
Epoch 19/21
34/34 [==============================] - 0s 1ms/step - loss: 0.0872 - accuracy: 0.9813
Epoch 20/21
34/34 [==============================] - 0s 1ms/step - loss: 0.0763 - accuracy: 0.9860
Epoch 21/21
3/3 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.5121951219512195
Confusion Matrix : 
 [[31 10]
 [10 31]]
Accuracy on test set:    0.7560975609756098
Sensitivity

32/32 [==============================] - 0s 1ms/step - loss: 0.0936 - accuracy: 0.9792
Epoch 19/21
32/32 [==============================] - 0s 1ms/step - loss: 0.0984 - accuracy: 0.9782
Epoch 20/21
32/32 [==============================] - 0s 1ms/step - loss: 0.0864 - accuracy: 0.9782
Epoch 21/21
5/5 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.6144738473249736
Confusion Matrix : 
 [[51 20]
 [ 8 63]]
Accuracy on test set:    0.8028169014084507
Sensitivity:    0.8873239436619719 	 Specificity:    0.7183098591549296
Precision  : 0.7590361445783133
Area Under Curve:    0.8028169014084507
(142, 1024) (142,)
(524, 1024) (52, 1024)
(12643, 1024) (3075, 1024)
Training Dataset
(1048, 1024) (1048,)
Validation Dataset
(104, 1024) (104,)




Epoch 1/21
33/33 [==============================] - 1s 2ms/step - loss: 0.6145 - accuracy: 0.6737
Epoch 2/21
33/33 [==============================] - 0s 2ms/step - loss: 0.5011 - accuracy: 0.7624
Epoch 3/21
33/33 

...

33/33 [==============================] - 0s 1ms/step - loss: 0.1184 - accuracy: 0.9687
Epoch 19/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1132 - accuracy: 0.9658
Epoch 20/21
33/33 [==============================] - 0s 1ms/step - loss: 0.0933 - accuracy: 0.9801
Epoch 21/21
4/4 [==============================] - 0s 926us/step
Independent Testing Matthews Correlation:  0.48154341234307674
Confusion Matrix : 
 [[39 11]
 [15 35]]
Accuracy on test set:    0.74
Sensitivity:    0.7 	 Specificity:    0.78
Precision  : 0.7608695652173914
Area Under Curve:    0.74
(100, 1024) (100,)
(528, 1024) (49, 1024)
(13953, 1024) (1808, 1024)
Training Dataset
(1056, 1024) (1056,)
Validation Dataset
(98, 1024) (98,)




Epoch 1/21
33/33 [==============================] - 1s 1ms/step - loss: 0.5987 - accuracy: 0.7112
Epoch 2/21
33/33 [==============================] - 0s 1ms/step - loss: 0.4757 - accuracy: 0.7888
Epoch 3/21
33/33 [==============================] - 0s 1ms/step - loss: 0

(530, 1024) (47, 1024)
(14418, 1024) (1343, 1024)
Training Dataset
(1060, 1024) (1060,)
Validation Dataset
(94, 1024) (94,)




Epoch 1/21
34/34 [==============================] - 1s 2ms/step - loss: 0.6026 - accuracy: 0.6858
Epoch 2/21
34/34 [==============================] - 0s 2ms/step - loss: 0.4808 - accuracy: 0.7877
Epoch 3/21
34/34 [==============================] - 0s 1ms/step - loss: 0.4248 - accuracy: 0.8255
Epoch 4/21
34/34 [==============================] - 0s 1ms/step - loss: 0.3803 - accuracy: 0.8425
Epoch 5/21
34/34 [==============================] - 0s 1ms/step - loss: 0.3568 - accuracy: 0.8623
Epoch 6/21
34/34 [==============================] - 0s 1ms/step - loss: 0.3119 - accuracy: 0.8783
Epoch 7/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2868 - accuracy: 0.8953
Epoch 8/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2773 - accuracy: 0.9047
Epoch 9/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2466 - accurac

Epoch 3/21
34/34 [==============================] - 0s 2ms/step - loss: 0.4085 - accuracy: 0.8355
Epoch 4/21
34/34 [==============================] - 0s 1ms/step - loss: 0.3645 - accuracy: 0.8544
Epoch 5/21
34/34 [==============================] - 0s 1ms/step - loss: 0.3205 - accuracy: 0.8837
Epoch 6/21
34/34 [==============================] - 0s 1ms/step - loss: 0.3006 - accuracy: 0.8847
Epoch 7/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2675 - accuracy: 0.9055
Epoch 8/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2402 - accuracy: 0.9282
Epoch 9/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2379 - accuracy: 0.9244
Epoch 10/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2094 - accuracy: 0.9282
Epoch 11/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2027 - accuracy: 0.9319
Epoch 12/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1816 - accuracy: 0.9414
Epoch 13/21
34/34

26/26 [==============================] - 0s 1ms/step - loss: 0.2159 - accuracy: 0.9280
Epoch 9/21
26/26 [==============================] - 0s 1ms/step - loss: 0.1968 - accuracy: 0.9500
Epoch 10/21
26/26 [==============================] - 0s 1ms/step - loss: 0.1702 - accuracy: 0.9524
Epoch 11/21
26/26 [==============================] - 0s 1ms/step - loss: 0.1559 - accuracy: 0.9622
Epoch 12/21
26/26 [==============================] - 0s 1ms/step - loss: 0.1416 - accuracy: 0.9622
Epoch 13/21
26/26 [==============================] - 0s 1ms/step - loss: 0.1278 - accuracy: 0.9695
Epoch 14/21
26/26 [==============================] - 0s 1ms/step - loss: 0.1150 - accuracy: 0.9744
Epoch 15/21
26/26 [==============================] - 0s 1ms/step - loss: 0.1029 - accuracy: 0.9720
Epoch 16/21
26/26 [==============================] - 0s 1ms/step - loss: 0.0935 - accuracy: 0.9793
Epoch 17/21
26/26 [==============================] - 0s 1ms/step - loss: 0.0799 - accuracy: 0.9829
Epoch 18/21
26/26 [====

33/33 [==============================] - 0s 1ms/step - loss: 0.1418 - accuracy: 0.9662
Epoch 14/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1192 - accuracy: 0.9691
Epoch 15/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1150 - accuracy: 0.9729
Epoch 16/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1069 - accuracy: 0.9720
Epoch 17/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1029 - accuracy: 0.9749
Epoch 18/21
33/33 [==============================] - 0s 1ms/step - loss: 0.0988 - accuracy: 0.9807
Epoch 19/21
33/33 [==============================] - 0s 1ms/step - loss: 0.0859 - accuracy: 0.9836
Epoch 20/21
33/33 [==============================] - 0s 1ms/step - loss: 0.0806 - accuracy: 0.9787
Epoch 21/21
4/4 [==============================] - 0s 958us/step
Independent Testing Matthews Correlation:  0.5095438042387856
Confusion Matrix : 
 [[50 13]
 [18 45]]
Accuracy on test set:    0.753968253968254
Sensitivit

In [9]:
from imblearn.under_sampling import RandomUnderSampler
a = random.sample(range(1, 1000000), 20)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        y_train_1 = tf.keras.utils.to_categorical(y_train,2)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        model = Sequential()

        model.add(Input(shape=(1024,)))

        model.add(Dense(64,activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(2,activation='softmax'))



        model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])

        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                        monitor = 'val_accuracy',
                                        verbose=0, 
                                        save_weights_only=False,
                                        save_best_only=True)

        reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=5, verbose=1, min_delta=1e-4, mode='max')

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

        history = model.fit(X_train, y_train_1, epochs=12,verbose=1, batch_size=32)

        Y_pred = model.predict(xval)
        Y_pred = (Y_pred > 0.5)
        y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
        y_pred = np.array(y_pred)

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("MLP Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(412, 1024) (168, 1024)
(11527, 1024) (4196, 1024)
Training Dataset
(824, 1024) (824,)
Validation Dataset
(336, 1024) (336,)




Epoch 1/12
26/26 [==============================] - 1s 2ms/step - loss: 0.6239 - accuracy: 0.6881
Epoch 2/12
26/26 [==============================] - 0s 1ms/step - loss: 0.4881 - accuracy: 0.7803
Epoch 3/12
26/26 [==============================] - 0s 1ms/step - loss: 0.4151 - accuracy: 0.8107
Epoch 4/12
26/26 [==============================] - 0s 1ms/step - loss: 0.3731 - accuracy: 0.8568
Epoch 5/12
26/26 [==============================] - 0s 1ms/step - loss: 0.3316 - accuracy: 0.8762
Epoch 6/12
26/26 [==============================] - 0s 1ms/step - loss: 0.2877 - accuracy: 0.8932
Epoch 7/12
26/26 [==============================] - 0s 1ms/step - loss: 0.2643 - accuracy: 0.9017
Epoch 8/12
26/26 [==============================] - 0s 1ms/step - loss: 0.2352 - accuracy: 0.9260
Epoch 9/12
26/26 [==============================] - 0s 1ms/step - loss: 0.2125 - accura

(529, 1024) (51, 1024)
(13657, 1024) (2066, 1024)
Training Dataset
(1058, 1024) (1058,)
Validation Dataset
(102, 1024) (102,)




Epoch 1/12
34/34 [==============================] - 1s 1ms/step - loss: 0.5926 - accuracy: 0.7079
Epoch 2/12
34/34 [==============================] - 0s 1ms/step - loss: 0.4671 - accuracy: 0.7911
Epoch 3/12
34/34 [==============================] - 0s 2ms/step - loss: 0.4035 - accuracy: 0.8280
Epoch 4/12
34/34 [==============================] - 0s 1ms/step - loss: 0.3631 - accuracy: 0.8658
Epoch 5/12
34/34 [==============================] - 0s 1ms/step - loss: 0.3283 - accuracy: 0.8790
Epoch 6/12
34/34 [==============================] - 0s 1ms/step - loss: 0.2993 - accuracy: 0.8819
Epoch 7/12
34/34 [==============================] - 0s 1ms/step - loss: 0.2748 - accuracy: 0.8979
Epoch 8/12
34/34 [==============================] - 0s 1ms/step - loss: 0.2477 - accuracy: 0.9130
Epoch 9/12
34/34 [==============================] - 0s 1ms/step - loss: 0.2336 - accur

3/3 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.50709255283711
Confusion Matrix : 
 [[30  6]
 [12 24]]
Accuracy on test set:    0.75
Sensitivity:    0.6666666666666666 	 Specificity:    0.8333333333333334
Precision  : 0.8
Area Under Curve:    0.75
(72, 1024) (72,)
(526, 1024) (51, 1024)
(14134, 1024) (1451, 1024)
Training Dataset
(1052, 1024) (1052,)
Validation Dataset
(102, 1024) (102,)




Epoch 1/12
33/33 [==============================] - 1s 1ms/step - loss: 0.5981 - accuracy: 0.7053
Epoch 2/12
33/33 [==============================] - 0s 2ms/step - loss: 0.4743 - accuracy: 0.7899
Epoch 3/12
33/33 [==============================] - 0s 1ms/step - loss: 0.4024 - accuracy: 0.8241
Epoch 4/12
33/33 [==============================] - 0s 1ms/step - loss: 0.3588 - accuracy: 0.8498
Epoch 5/12
33/33 [==============================] - 0s 1ms/step - loss: 0.3317 - accuracy: 0.8764
Epoch 6/12
33/33 [==============================] - 0s 1ms/step - l

3/3 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.5170876899950191
Confusion Matrix : 
 [[32  7]
 [12 27]]
Accuracy on test set:    0.7564102564102564
Sensitivity:    0.6923076923076923 	 Specificity:    0.8205128205128205
Precision  : 0.7941176470588235
Area Under Curve:    0.7564102564102564
(78, 1024) (78,)
(487, 1024) (90, 1024)
(11969, 1024) (3616, 1024)
Training Dataset
(974, 1024) (974,)
Validation Dataset
(180, 1024) (180,)




Epoch 1/12
31/31 [==============================] - 1s 2ms/step - loss: 0.6196 - accuracy: 0.6694
Epoch 2/12
31/31 [==============================] - 0s 1ms/step - loss: 0.4965 - accuracy: 0.7741
Epoch 3/12
31/31 [==============================] - 0s 2ms/step - loss: 0.4305 - accuracy: 0.8080
Epoch 4/12
31/31 [==============================] - 0s 1ms/step - loss: 0.3905 - accuracy: 0.8244
Epoch 5/12
31/31 [==============================] - 0s 1ms/step - loss: 0.3668 - accuracy: 0.8450
Epoch 6/12
31/31 [======

33/33 [==============================] - 0s 1ms/step - loss: 0.2537 - accuracy: 0.9067
Epoch 9/12
33/33 [==============================] - 0s 1ms/step - loss: 0.2242 - accuracy: 0.9210
Epoch 10/12
33/33 [==============================] - 0s 1ms/step - loss: 0.2023 - accuracy: 0.9362
Epoch 11/12
33/33 [==============================] - 0s 1ms/step - loss: 0.1866 - accuracy: 0.9390
Epoch 12/12
4/4 [==============================] - 0s 918us/step
Independent Testing Matthews Correlation:  0.5243041203911968
Confusion Matrix : 
 [[45  9]
 [17 37]]
Accuracy on test set:    0.7592592592592593
Sensitivity:    0.6851851851851852 	 Specificity:    0.8333333333333334
Precision  : 0.8043478260869565
Area Under Curve:    0.7592592592592593
(108, 1024) (108,)
(540, 1024) (39, 1024)
(14094, 1024) (1681, 1024)
Training Dataset
(1080, 1024) (1080,)
Validation Dataset
(78, 1024) (78,)




Epoch 1/12
34/34 [==============================] - 1s 1ms/step - loss: 0.6016 - accuracy: 0.7019
Epoch 2/12
34/34 

...

26/26 [==============================] - 1s 1ms/step - loss: 0.5852 - accuracy: 0.7252
Epoch 2/12
26/26 [==============================] - 0s 1ms/step - loss: 0.4487 - accuracy: 0.8051
Epoch 3/12
26/26 [==============================] - 0s 1ms/step - loss: 0.3817 - accuracy: 0.8414
Epoch 4/12
26/26 [==============================] - 0s 1ms/step - loss: 0.3379 - accuracy: 0.8705
Epoch 5/12
26/26 [==============================] - 0s 1ms/step - loss: 0.2936 - accuracy: 0.8959
Epoch 6/12
26/26 [==============================] - 0s 1ms/step - loss: 0.2672 - accuracy: 0.9116
Epoch 7/12
26/26 [==============================] - 0s 1ms/step - loss: 0.2393 - accuracy: 0.9225
Epoch 8/12
26/26 [==============================] - 0s 1ms/step - loss: 0.2087 - accuracy: 0.9322
Epoch 9/12
26/26 [==============================] - 0s 1ms/step - loss: 0.1897 - accuracy: 0.9443
Epoch 10/12
26/26 [==============================] - 0s 1ms/step - loss: 0.1701 - accuracy: 0.9455
Epoch 11/12
26/26 [===========

(536, 1024) (39, 1024)
(13700, 1024) (1927, 1024)
Training Dataset
(1072, 1024) (1072,)
Validation Dataset
(78, 1024) (78,)




Epoch 1/12
34/34 [==============================] - 1s 1ms/step - loss: 0.5901 - accuracy: 0.7257
Epoch 2/12
34/34 [==============================] - 0s 1ms/step - loss: 0.4655 - accuracy: 0.7920
Epoch 3/12
34/34 [==============================] - 0s 1ms/step - loss: 0.4033 - accuracy: 0.8265
Epoch 4/12
34/34 [==============================] - 0s 1ms/step - loss: 0.3442 - accuracy: 0.8685
Epoch 5/12
34/34 [==============================] - 0s 1ms/step - loss: 0.3156 - accuracy: 0.8853
Epoch 6/12
34/34 [==============================] - 0s 1ms/step - loss: 0.2770 - accuracy: 0.8993
Epoch 7/12
34/34 [==============================] - 0s 1ms/step - loss: 0.2543 - accuracy: 0.9114
Epoch 8/12
34/34 [==============================] - 0s 1ms/step - loss: 0.2356 - accuracy: 0.9151
Epoch 9/12
34/34 [==============================] - 0s 1ms/step - loss: 0.2081 - accurac

(512, 1024) (63, 1024)
(13604, 1024) (2023, 1024)
Training Dataset
(1024, 1024) (1024,)
Validation Dataset
(126, 1024) (126,)




Epoch 1/12
32/32 [==============================] - 1s 1ms/step - loss: 0.6047 - accuracy: 0.7236
Epoch 2/12
32/32 [==============================] - 0s 1ms/step - loss: 0.4755 - accuracy: 0.7949
Epoch 3/12
32/32 [==============================] - 0s 1ms/step - loss: 0.4071 - accuracy: 0.8145
Epoch 4/12
32/32 [==============================] - 0s 1ms/step - loss: 0.3534 - accuracy: 0.8584
Epoch 5/12
32/32 [==============================] - 0s 1ms/step - loss: 0.3088 - accuracy: 0.8799
Epoch 6/12
32/32 [==============================] - 0s 1ms/step - loss: 0.2870 - accuracy: 0.8838
Epoch 7/12
32/32 [==============================] - 0s 1ms/step - loss: 0.2562 - accuracy: 0.8955
Epoch 8/12
32/32 [==============================] - 0s 1ms/step - loss: 0.2249 - accuracy: 0.9219
Epoch 9/12
32/32 [==============================] - 0s 1ms/step - loss: 0.2066 - accur

3/3 [==============================] - 0s 991us/step
Independent Testing Matthews Correlation:  0.5962847939999439
Confusion Matrix : 
 [[27 15]
 [ 3 39]]
Accuracy on test set:    0.7857142857142857
Sensitivity:    0.9285714285714286 	 Specificity:    0.6428571428571429
Precision  : 0.7222222222222222
Area Under Curve:    0.7857142857142857
(84, 1024) (84,)
(542, 1024) (38, 1024)
(14165, 1024) (1540, 1024)
Training Dataset
(1084, 1024) (1084,)
Validation Dataset
(76, 1024) (76,)




Epoch 1/12
34/34 [==============================] - 1s 2ms/step - loss: 0.5656 - accuracy: 0.7334
Epoch 2/12
34/34 [==============================] - 0s 1ms/step - loss: 0.4316 - accuracy: 0.8127
Epoch 3/12
34/34 [==============================] - 0s 1ms/step - loss: 0.3751 - accuracy: 0.8423
Epoch 4/12
34/34 [==============================] - 0s 2ms/step - loss: 0.3250 - accuracy: 0.8745
Epoch 5/12
34/34 [==============================] - 0s 1ms/step - loss: 0.2922 - accuracy: 0.8911
Epoch 6/12
34/34 [====

34/34 [==============================] - 0s 1ms/step - loss: 0.1752 - accuracy: 0.9478
Epoch 12/12
3/3 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.34827259515942904
Confusion Matrix : 
 [[34 10]
 [19 25]]
Accuracy on test set:    0.6704545454545454
Sensitivity:    0.5681818181818182 	 Specificity:    0.7727272727272727
Precision  : 0.7142857142857143
Area Under Curve:    0.6704545454545455
(88, 1024) (88,)
(526, 1024) (54, 1024)
(14283, 1024) (1422, 1024)
Training Dataset
(1052, 1024) (1052,)
Validation Dataset
(108, 1024) (108,)




Epoch 1/12
33/33 [==============================] - 1s 1ms/step - loss: 0.6078 - accuracy: 0.6968
Epoch 2/12
33/33 [==============================] - 0s 1ms/step - loss: 0.4871 - accuracy: 0.7795
Epoch 3/12
33/33 [==============================] - 0s 1ms/step - loss: 0.4205 - accuracy: 0.8156
Epoch 4/12
33/33 [==============================] - 0s 1ms/step - loss: 0.3748 - accuracy: 0.8527
Epoch 5/12
33/33 [==

# Logistic Regression 10 Fold CV

In [4]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
a = random.sample(range(1, 1000000), 3)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = LogisticRegression(penalty='l2',solver='lbfgs',random_state=seed)

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("LR Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("LR Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(515, 1024) (64, 1024)
(13880, 1024) (1770, 1024)
Training Dataset
(1030, 1024) (1030,)
Validation Dataset
(128, 1024) (128,)






/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Matthews Correlation :  0.6011333973099161
Confusion Matrix : 
 [[46 18]
 [ 8 56]]
Accuracy on test set:    0.796875
Sensitivity:    0.875 	 Specificity:    0.71875
Area Under Curve:    0.796875
(541, 1024) (38, 1024)
(14090, 1024) (1560, 1024)
Training Dataset
(1082, 1024) (1082,)
Validation Dataset
(76, 1024) (76,)




Matthews Correlation :  0.5574784668619939
Confusion Matrix : 
 [[32  6]
 [11 27]]
Accuracy on test set:    0.7763157894736842
Sensitivity:    0.7105263157894737 	 Specificity:    0.8421052631578947
Area Under Curve:    0.7763157894736842


/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(526, 1024) (53, 1024)
(14301, 1024) (1349, 1024)
Training Dataset
(1052, 1024) (1052,)
Validation Dataset
(106, 1024) (106,)






...

/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(536, 1024) (42, 1024)
(14150, 1024) (1524, 1024)
Training Dataset
(1072, 1024) (1072,)
Validation Dataset
(84, 1024) (84,)




Matthews Correlation :  0.4048766819951098
Confusion Matrix : 
 [[30 12]
 [13 29]]
Accuracy on test set:    0.7023809523809523
Sensitivity:    0.6904761904761905 	 Specificity:    0.7142857142857143
Area Under Curve:    0.7023809523809524


/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(526, 1024) (52, 1024)
(12620, 1024) (3054, 1024)
Training Dataset
(1052, 1024) (1052,)
Validation Dataset
(104, 1024) (104,)




Matthews Correlation :  0.3993615319154359
Confusion Matrix : 
 [[43  9]
 [23 29]]
Accuracy on test set:    0.6923076923076923
Sensitivity:    0.5576923076923077 	 Specificity:    0.8269230769230769
Area Under Curve:    0.6923076923076923
LR Mean MCC = 0.4432 ± 0.1216 and Mean AUC = 0.7169 ± 0.0636
LR Mean MCC:    0.44318845992029304
Mean Sensitivity = 0.6553 ± 0.1665 and Mean Specificity = 0.7785 ± 0.0662 and Mean Accuracy = 0.7169 ± 0.0636
Mean Precision = 0.7450 ± 0.0512 
















/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Random Forest 10 Fold CV

In [10]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
a = random.sample(range(1, 1000000), 4)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = RandomForestClassifier(max_depth=10, random_state=seed)

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("RF Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("RF Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(514, 1024) (55, 1024)
(12973, 1024) (904, 1024)
Training Dataset
(1028, 1024) (1028,)
Validation Dataset
(110, 1024) (110,)




Matthews Correlation :  0.27565892320998564
Confusion Matrix : 
 [[39 16]
 [24 31]]
Accuracy on test set:    0.6363636363636364
Sensitivity:    0.5636363636363636 	 Specificity:    0.7090909090909091
Area Under Curve:    0.6363636363636364
(529, 1024) (40, 1024)
(12178, 1024) (1699, 1024)
Training Dataset
(1058, 1024) (1058,)
Validation Dataset
(80, 1024) (80,)




Matthews Correlation :  0.5006261743217589
Confusion Matrix : 
 [[29 11]
 [ 9 31]]
Accuracy on test set:    0.75
Sensitivity:    0.775 	 Specificity:    0.725
Area Under Curve:    0.75
(518, 1024) (51, 1024)
(12217, 1024) (1660, 1024)
Training Dataset
(1036, 1024) (1036,)
Validation Dataset
(102, 1024) (102,)




Matthews Correlation :  0.6120938403510622
Confusion Matrix : 
 [[44  7]
 [13 38]]
Accuracy on test set:    0.803921568627451
Sensitivity:    0.7450980392156863 	 Specificity:    0.8627450

Matthews Correlation :  0.584063870042052
Confusion Matrix : 
 [[49 11]
 [14 46]]
Accuracy on test set:    0.7916666666666666
Sensitivity:    0.7666666666666667 	 Specificity:    0.8166666666666667
Area Under Curve:    0.7916666666666666
(427, 1024) (146, 1024)
(13539, 1024) (1822, 1024)
Training Dataset
(854, 1024) (854,)
Validation Dataset
(292, 1024) (292,)




Matthews Correlation :  0.17522838380184286
Confusion Matrix : 
 [[ 70  76]
 [ 45 101]]
Accuracy on test set:    0.5856164383561644
Sensitivity:    0.6917808219178082 	 Specificity:    0.4794520547945205
Area Under Curve:    0.5856164383561644
(511, 1024) (62, 1024)
(14125, 1024) (1236, 1024)
Training Dataset
(1022, 1024) (1022,)
Validation Dataset
(124, 1024) (124,)




Matthews Correlation :  0.4752821235485438
Confusion Matrix : 
 [[51 11]
 [22 40]]
Accuracy on test set:    0.7338709677419355
Sensitivity:    0.6451612903225806 	 Specificity:    0.8225806451612904
Area Under Curve:    0.7338709677419354
(511, 1024) (62, 102

# XGBoost 10 Fold CV

In [6]:
# pip install xgboost

In [7]:
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler

a = random.sample(range(1, 1000000), 20)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = XGBClassifier()

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("XGB Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("XGB Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(415, 1024) (155, 1024)
(14153, 1024) (1537, 1024)
Training Dataset
(830, 1024) (830,)
Validation Dataset
(310, 1024) (310,)




Matthews Correlation :  0.20680051997471113
Confusion Matrix : 
 [[98 57]
 [66 89]]
Accuracy on test set:    0.603225806451613
Sensitivity:    0.5741935483870968 	 Specificity:    0.632258064516129
Area Under Curve:    0.603225806451613
(508, 1024) (62, 1024)
(14269, 1024) (1421, 1024)
Training Dataset
(1016, 1024) (1016,)
Validation Dataset
(124, 1024) (124,)




Matthews Correlation :  0.6465081838352359
Confusion Matrix : 
 [[49 13]
 [ 9 53]]
Accuracy on test set:    0.8225806451612904
Sensitivity:    0.8548387096774194 	 Specificity:    0.7903225806451613
Area Under Curve:    0.8225806451612904
(511, 1024) (59, 1024)
(14023, 1024) (1667, 1024)
Training Dataset
(1022, 1024) (1022,)
Validation Dataset
(118, 1024) (118,)




Matthews Correlation :  0.5443310539518174
Confusion Matrix : 
 [[48 11]
 [16 43]]
Accuracy on test set:    0.7711864406779662
Sensitiv

(540, 1024) (39, 1024)
(13771, 1024) (1911, 1024)
Training Dataset
(1080, 1024) (1080,)
Validation Dataset
(78, 1024) (78,)




Matthews Correlation :  0.427617987059879
Confusion Matrix : 
 [[33  6]
 [17 22]]
Accuracy on test set:    0.7051282051282052
Sensitivity:    0.5641025641025641 	 Specificity:    0.8461538461538461
Area Under Curve:    0.7051282051282051
(538, 1024) (41, 1024)
(12152, 1024) (3530, 1024)
Training Dataset
(1076, 1024) (1076,)
Validation Dataset
(82, 1024) (82,)




Matthews Correlation :  0.47784568208646405
Confusion Matrix : 
 [[35  6]
 [16 25]]
Accuracy on test set:    0.7317073170731707
Sensitivity:    0.6097560975609756 	 Specificity:    0.8536585365853658
Area Under Curve:    0.7317073170731707
(502, 1024) (77, 1024)
(14138, 1024) (1544, 1024)
Training Dataset
(1004, 1024) (1004,)
Validation Dataset
(154, 1024) (154,)




Matthews Correlation :  0.5150787536377128
Confusion Matrix : 
 [[65 12]
 [26 51]]
Accuracy on test set:    0.7532467532467533
Sensitivi

Matthews Correlation :  0.6174654780291237
Confusion Matrix : 
 [[43 14]
 [ 8 49]]
Accuracy on test set:    0.8070175438596491
Sensitivity:    0.8596491228070176 	 Specificity:    0.7543859649122807
Area Under Curve:    0.8070175438596491
(521, 1024) (58, 1024)
(14832, 1024) (899, 1024)
Training Dataset
(1042, 1024) (1042,)
Validation Dataset
(116, 1024) (116,)




Matthews Correlation :  0.5006701916776904
Confusion Matrix : 
 [[45 13]
 [16 42]]
Accuracy on test set:    0.75
Sensitivity:    0.7241379310344828 	 Specificity:    0.7758620689655172
Area Under Curve:    0.75
XGB Mean MCC = 0.5313 ± 0.1122 and Mean AUC = 0.7642 ± 0.0560
XGB Mean MCC:    0.5313423511671822
Mean Sensitivity = 0.7402 ± 0.0868 and Mean Specificity = 0.7881 ± 0.0585 and Mean Accuracy = 0.7642 ± 0.0560
Mean Precision = 0.7780 ± 0.0548 














(518, 1024) (55, 1024)
(14204, 1024) (1383, 1024)
Training Dataset
(1036, 1024) (1036,)
Validation Dataset
(110, 1024) (110,)




Matthews Correlation :  0.532940584

(517, 1024) (62, 1024)
(12286, 1024) (3151, 1024)
Training Dataset
(1034, 1024) (1034,)
Validation Dataset
(124, 1024) (124,)




Matthews Correlation :  0.5809475019311126
Confusion Matrix : 
 [[50 12]
 [14 48]]
Accuracy on test set:    0.7903225806451613
Sensitivity:    0.7741935483870968 	 Specificity:    0.8064516129032258
Area Under Curve:    0.7903225806451611
(547, 1024) (32, 1024)
(13838, 1024) (1599, 1024)
Training Dataset
(1094, 1024) (1094,)
Validation Dataset
(64, 1024) (64,)




Matthews Correlation :  0.5809475019311126
Confusion Matrix : 
 [[21 11]
 [ 3 29]]
Accuracy on test set:    0.78125
Sensitivity:    0.90625 	 Specificity:    0.65625
Area Under Curve:    0.78125
(538, 1024) (41, 1024)
(13862, 1024) (1575, 1024)
Training Dataset
(1076, 1024) (1076,)
Validation Dataset
(82, 1024) (82,)




Matthews Correlation :  0.5281452275692498
Confusion Matrix : 
 [[36  5]
 [15 26]]
Accuracy on test set:    0.7560975609756098
Sensitivity:    0.6341463414634146 	 Specificity:    

Matthews Correlation :  0.48107977297846943
Confusion Matrix : 
 [[57 16]
 [22 51]]
Accuracy on test set:    0.7397260273972602
Sensitivity:    0.6986301369863014 	 Specificity:    0.7808219178082192
Area Under Curve:    0.7397260273972603
(506, 1024) (69, 1024)
(14550, 1024) (830, 1024)
Training Dataset
(1012, 1024) (1012,)
Validation Dataset
(138, 1024) (138,)




Matthews Correlation :  0.37685117317409145
Confusion Matrix : 
 [[47 22]
 [21 48]]
Accuracy on test set:    0.6884057971014492
Sensitivity:    0.6956521739130435 	 Specificity:    0.6811594202898551
Area Under Curve:    0.6884057971014492
(546, 1024) (29, 1024)
(14113, 1024) (1267, 1024)
Training Dataset
(1092, 1024) (1092,)
Validation Dataset
(58, 1024) (58,)




Matthews Correlation :  0.6301260378126045
Confusion Matrix : 
 [[26  3]
 [ 8 21]]
Accuracy on test set:    0.8103448275862069
Sensitivity:    0.7241379310344828 	 Specificity:    0.896551724137931
Area Under Curve:    0.8103448275862069
(534, 1024) (41, 1024)
(1

(429, 1024) (151, 1024)
(13781, 1024) (1995, 1024)
Training Dataset
(858, 1024) (858,)
Validation Dataset
(302, 1024) (302,)




Matthews Correlation :  0.31150428894227306
Confusion Matrix : 
 [[ 96  55]
 [ 49 102]]
Accuracy on test set:    0.6556291390728477
Sensitivity:    0.6754966887417219 	 Specificity:    0.6357615894039735
Area Under Curve:    0.6556291390728477
XGB Mean MCC = 0.4587 ± 0.0678 and Mean AUC = 0.7263 ± 0.0320
XGB Mean MCC:    0.45873081422166706
Mean Sensitivity = 0.7268 ± 0.0786 and Mean Specificity = 0.7257 ± 0.0905 and Mean Accuracy = 0.7263 ± 0.0320
Mean Precision = 0.7338 ± 0.0612 














(537, 1024) (43, 1024)
(13948, 1024) (1497, 1024)
Training Dataset
(1074, 1024) (1074,)
Validation Dataset
(86, 1024) (86,)




Matthews Correlation :  0.6599663291074443
Confusion Matrix : 
 [[32 11]
 [ 4 39]]
Accuracy on test set:    0.8255813953488372
Sensitivity:    0.9069767441860465 	 Specificity:    0.7441860465116279
Area Under Curve:    0.8255813953488371
(51

(505, 1024) (68, 1024)
(14343, 1024) (1329, 1024)
Training Dataset
(1010, 1024) (1010,)
Validation Dataset
(136, 1024) (136,)




Matthews Correlation :  0.47387910220727386
Confusion Matrix : 
 [[46 22]
 [14 54]]
Accuracy on test set:    0.7352941176470589
Sensitivity:    0.7941176470588235 	 Specificity:    0.6764705882352942
Area Under Curve:    0.7352941176470589
(531, 1024) (42, 1024)
(14726, 1024) (946, 1024)
Training Dataset
(1062, 1024) (1062,)
Validation Dataset
(84, 1024) (84,)




Matthews Correlation :  0.5262013605584339
Confusion Matrix : 
 [[30 12]
 [ 8 34]]
Accuracy on test set:    0.7619047619047619
Sensitivity:    0.8095238095238095 	 Specificity:    0.7142857142857143
Area Under Curve:    0.761904761904762
(527, 1024) (46, 1024)
(12516, 1024) (3156, 1024)
Training Dataset
(1054, 1024) (1054,)
Validation Dataset
(92, 1024) (92,)




Matthews Correlation :  0.6236095644623236
Confusion Matrix : 
 [[42  4]
 [14 32]]
Accuracy on test set:    0.8043478260869565
Sensitivit

(528, 1024) (51, 1024)
(14929, 1024) (652, 1024)
Training Dataset
(1056, 1024) (1056,)
Validation Dataset
(102, 1024) (102,)




Matthews Correlation :  0.5823232315653921
Confusion Matrix : 
 [[31 20]
 [ 3 48]]
Accuracy on test set:    0.7745098039215687
Sensitivity:    0.9411764705882353 	 Specificity:    0.6078431372549019
Area Under Curve:    0.7745098039215687
(536, 1024) (43, 1024)
(13678, 1024) (1903, 1024)
Training Dataset
(1072, 1024) (1072,)
Validation Dataset
(86, 1024) (86,)




Matthews Correlation :  0.39925468345586773
Confusion Matrix : 
 [[33 10]
 [16 27]]
Accuracy on test set:    0.6976744186046512
Sensitivity:    0.627906976744186 	 Specificity:    0.7674418604651163
Area Under Curve:    0.6976744186046512
(529, 1024) (50, 1024)
(13879, 1024) (1702, 1024)
Training Dataset
(1058, 1024) (1058,)
Validation Dataset
(100, 1024) (100,)




Matthews Correlation :  0.48221102489208245
Confusion Matrix : 
 [[44  6]
 [21 29]]
Accuracy on test set:    0.73
Sensitivity:    0.58 

(533, 1024) (45, 1024)
(14753, 1024) (963, 1024)
Training Dataset
(1066, 1024) (1066,)
Validation Dataset
(90, 1024) (90,)




Matthews Correlation :  0.5813776741499453
Confusion Matrix : 
 [[33 12]
 [ 7 38]]
Accuracy on test set:    0.7888888888888889
Sensitivity:    0.8444444444444444 	 Specificity:    0.7333333333333333
Area Under Curve:    0.788888888888889
(516, 1024) (62, 1024)
(14472, 1024) (1244, 1024)
Training Dataset
(1032, 1024) (1032,)
Validation Dataset
(124, 1024) (124,)




Matthews Correlation :  0.6943617218584767
Confusion Matrix : 
 [[54  8]
 [11 51]]
Accuracy on test set:    0.8467741935483871
Sensitivity:    0.8225806451612904 	 Specificity:    0.8709677419354839
Area Under Curve:    0.8467741935483871
(414, 1024) (164, 1024)
(12152, 1024) (3564, 1024)
Training Dataset
(828, 1024) (828,)
Validation Dataset
(328, 1024) (328,)




Matthews Correlation :  0.3676073110469039
Confusion Matrix : 
 [[120  44]
 [ 60 104]]
Accuracy on test set:    0.6829268292682927
Sensit

# SVM 10 Fold CV

In [8]:
from sklearn import svm
from imblearn.under_sampling import RandomUnderSampler

a = random.sample(range(1, 1000000), 1)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = svm.SVC(C=10, kernel='rbf')

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("SVM Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("SVM Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(530, 1024) (48, 1024)
(13820, 1024) (1923, 1024)
Training Dataset
(1060, 1024) (1060,)
Validation Dataset
(96, 1024) (96,)




Matthews Correlation :  0.7089490077940542
Confusion Matrix : 
 [[42  6]
 [ 8 40]]
Accuracy on test set:    0.8541666666666666
Sensitivity:    0.8333333333333334 	 Specificity:    0.875
Area Under Curve:    0.8541666666666667
(518, 1024) (60, 1024)
(13951, 1024) (1792, 1024)
Training Dataset
(1036, 1024) (1036,)
Validation Dataset
(120, 1024) (120,)




Matthews Correlation :  0.45006251302384814
Confusion Matrix : 
 [[44 16]
 [17 43]]
Accuracy on test set:    0.725
Sensitivity:    0.7166666666666667 	 Specificity:    0.7333333333333333
Area Under Curve:    0.7250000000000001
(539, 1024) (39, 1024)
(14496, 1024) (1247, 1024)
Training Dataset
(1078, 1024) (1078,)
Validation Dataset
(78, 1024) (78,)




Matthews Correlation :  0.36196138299651337
Confusion Matrix : 
 [[29 10]
 [15 24]]
Accuracy on test set:    0.6794871794871795
Sensitivity:    0.615384615384615

# Thank You